In [7]:
import pickle
import numpy as np
import gc; gc.enable()
import os

In [8]:
SAVE_FOLDER = 'HASH_CHAR_WB_23/'

In [3]:
with open('text_feature.pickle', 'rb') as handle:
    text_feature = pickle.load(handle)

In [4]:
if not os.path.exists(SAVE_FOLDER):
    os.makedirs(SAVE_FOLDER)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer

In [6]:
hash_char_wb_features = HashingVectorizer(ngram_range=(2, 3), 
                                          analyzer='char_wb',
                                          n_features=2**18,
                                          norm='l2', 
                                          dtype=np.float32).fit_transform(text_feature.values)
print(hash_char_wb_features.shape)

(2011862, 262144)


In [7]:
with open(SAVE_FOLDER+'hash_char_wb_features.pickle', 'wb') as handle:
    pickle.dump(hash_char_wb_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
del hash_char_wb_features; gc.collect()

103

## Merge

In [9]:
import pickle
import pandas as pd
import numpy as np
import gc; gc.enable()
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack, save_npz

In [10]:
comm_feature = 'all_features.pickle' 
text_feature = SAVE_FOLDER+'hash_char_wb_features.pickle'

In [11]:
with open(comm_feature, 'rb') as handle1:
    comm_feature = pickle.load(handle1).astype(np.float32)
    print(type(comm_feature))
    
with open(text_feature, 'rb') as handle2:
    text_feature = pickle.load(handle2)
    print(type(text_feature))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>


In [12]:
all_features = hstack([comm_feature, text_feature]).tocsr()

In [13]:
all_features = all_features.astype(np.float64)

In [14]:
all_features.shape

(2011862, 357887)

In [15]:
save_npz('all_features_hv_charwb23.npz', all_features)